In [1]:
import dill
import numpy as np
from igm_emulator.emulator.plotVis import v_bins
import h5py
from progressbar import ProgressBar
'''
load model and auto-corr
'''
redshift = 5.4

# get the appropriate string and pathlength for chosen redshift
zs = np.array([5.4, 5.5, 5.6, 5.7, 5.8, 5.9, 6.0])
z_idx = np.argmin(np.abs(zs - redshift))
z_strings = ['z54', 'z55', 'z56', 'z57', 'z58', 'z59', 'z6']
z_string = z_strings[z_idx]
in_path_hdf5 = '/home/zhenyujin/igm_emulator/igm_emulator/emulator/best_params/'
f = h5py.File(in_path_hdf5 + f'z{redshift}_nn_savefile.hdf5', 'r')
emu_name = f'{z_string}_best_param_training_768.p'

in_path = f'/mnt/quasar2/mawolfson/correlation_funct/temp_gamma/final/{z_string}/final_135/'
n_paths = np.array([17, 16, 16, 15, 15, 15, 14]) #skewers_per_data
n_path = n_paths[z_idx]
vbins = v_bins
param_in_path = '/mnt/quasar2/mawolfson/correlation_funct/temp_gamma/final/'
param_dict = dill.load(open(param_in_path + f'{z_string}_params.p', 'rb'))

fobs = param_dict['fobs']  # average observed flux <F> ~ Gamma_HI
log_T0s = param_dict['log_T0s']  # log(T_0) from temperature - density relation
T0s = np.power(10,log_T0s)
gammas = param_dict['gammas']  # gamma from temperature - density relation

best_params = dill.load(open(in_path_hdf5 + emu_name, 'rb'))
T0_idx = 11 #0-14
g_idx = 4 #0-8
f_idx = 7 #0-8

one_cov_name = "z54_data_nearest_model_set_bins_4_log_like_on_grid_5_one_prior_T11_G4_F7_R_30000_one_covariance.hdf5"
one_cov_dict = h5py.File(in_path + one_cov_name, 'r')
like_name = f'likelihood_dicts_R_30000_nf_9_T{T0_idx}_G{g_idx}_SNR0_F{f_idx}_ncovar_500000_P{n_path}_set_bins_4.p'
like_dict = dill.load(open(in_path + like_name, 'rb'))
print(one_cov_dict.keys())
print('...')
print(like_dict.keys())

mock_name = f'mocks_R_30000_nf_9_T{T0_idx}_G{g_idx}_SNR0_F{f_idx}_P{n_path}_set_bins_4.p'
mocks = dill.load(open(in_path + mock_name, 'rb'))
theta_true = [fobs[f_idx], T0s[T0_idx], gammas[g_idx]]

mock_flux = mocks[0:5,:]
mean_flux = like_dict['mean_data']

<KeysViewHDF5 ['fobs_grid', 'gammas_grid', 'log_likelihood_grid', 'temps_grid', 'true_theta']>
...
dict_keys(['mean_data', 'covariance', 'correlation', 'condition_number', 'inv_covariance', 'sign', 'log_determinant'])


In [2]:
true_theta = np.array(one_cov_dict['true_theta'])
print(true_theta)
fobs_grid = one_cov_dict['fobs_grid']
gammas_grid = one_cov_dict['gammas_grid']
temps_grid = one_cov_dict['temps_grid']
print(fobs_grid,temps_grid,gammas_grid)

molly_loglike_grid = one_cov_dict['log_likelihood_grid']
print(molly_loglike_grid)

[[1.41775714e+04 1.35200000e+00 9.90000000e-02]
 [1.41775714e+04 1.35200000e+00 9.90000000e-02]
 [1.41775714e+04 1.35200000e+00 9.90000000e-02]
 [1.41775714e+04 1.35200000e+00 9.90000000e-02]
 [1.41775714e+04 1.35200000e+00 9.90000000e-02]]
<HDF5 dataset "fobs_grid": shape (33,), type "<f8"> <HDF5 dataset "temps_grid": shape (57,), type "<f8"> <HDF5 dataset "gammas_grid": shape (33,), type "<f8">
<HDF5 dataset "log_likelihood_grid": shape (5, 57, 33, 33), type "<f8">


In [3]:
from nn_hmc_3d_x import NN_HMC_X

nn_x = NN_HMC_X(vbins,best_params,T0s,gammas,fobs,like_dict)
x_true = nn_x.theta_to_x(theta_true)

Layers: [100, 100, 100, 276]
Activation: <function leaky_relu at 0x7f8b94080ae8>
L2 regularization lambda: 0.0001
meanX = [8.400e-02 9.149e+03 1.352e+00]
stdX = [1.36930639e-02 5.50374022e+03 6.02494813e-01]
train: (768, 3)
(358, 276)


In [ ]:
n_inference = 0
linda_loglike_grid = np.empty([n_inference, len(fobs_grid), len(temps_grid), len(gammas_grid)])
pbar = ProgressBar()

for mock_idx in pbar(range(n_inference)):
    flux = mocks[mock_idx, :]
    print("START RUNNING")
    for f_plot_idx, f_plot in enumerate(fobs_grid):
        for t_plot_idx, t_plot in enumerate(temps_grid):
                for g_plot_idx, g_plot in enumerate(gammas_grid):
                        linda_loglike_grid[mock_idx, f_plot_idx, t_plot_idx, g_plot_idx] =  nn_x.log_likelihood((f_plot, t_plot, g_plot), flux)

In [ ]:
    x_size = 3.5
    dpi_value = 200
    plt_params = {'legend.fontsize': 7,
                  'legend.frameon': False,
                  'axes.labelsize': 8,
                  'axes.titlesize': 6.5,
                  'figure.titlesize': 8,
                  'xtick.labelsize': 7,
                  'ytick.labelsize': 7,
                  'lines.linewidth': 1,
                  'lines.markersize': 2,
                  'errorbar.capsize': 3,
                  'font.family': 'serif',
                  # 'text.usetex': True,
                  'xtick.minor.visible': True,
                  }
    plt.rcParams.update(plt_params)

    # plot one 1d slice - temps only
    g_plot_idx = int(np.floor(len(gammas_grid)/2.))
    f_plot_idx = int(np.floor(len(fobs_grid)/2.))

    # IPython.embed()

    slice_fig = plt.figure(figsize=(x_size, x_size*.77*5.*.5), constrained_layout=True,
                                    dpi=dpi_value,
                                    )
    # one_correlation_fig.set_constrained_layout_pads(w_pad=0, h_pad=0, hspace=0, wspace=0)
    grid = slice_fig.add_gridspec(
        nrows=5, ncols=1, # width_ratios=[20, 20, 20, 20, 20, 1],
    )

    for mock_idx in range(n_inference):
        axes = slice_fig.add_subplot(grid[mock_idx])

        axes.plot(temps_grid, linda_loglike_grid[mock_idx, f_plot_idx, :, g_plot_idx])

        axes.set_ylabel('log(likelihood)')
        axes.set_title(f'mock {mock_idx}')

    axes.set_xlabel('$T_0$ (K)')